<a href="https://colab.research.google.com/github/Aphilak660510747/229352-stat/blob/main/Lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #4

In [20]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from scipy.stats import uniform

In [21]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


### Naive Bayes [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

In [22]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

### Random Search Cross-Validation [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

### Uniform distribution in `Scipy` [(Documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html)

SyntaxError: incomplete input (ipython-input-30-2274929817.py, line 1)

#### Exercise

1. For the Naive Bayes model, use grid search 5-fold cross-validation across different values of `alpha` to find the best model.

2. For the best value of `alpha`, compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* What is the model's `f1_macro` score?

3. Repeat Exercise 1 and 2 for **random search** 5-fold cross validation across different values of `alpha`. Compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* Did you get a better `f1_macro` score compared to grid search in Exercise 2?

In [31]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import f1_score

# 1. Load the dataset (example: 20 Newsgroups)
# We'll load a subset for quicker demonstration.
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
print("Loading 20 Newsgroups dataset...")
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
print("Dataset loaded.")

X_train, y_train = newsgroups_train.data, newsgroups_train.target
X_test, y_test = newsgroups_test.data, newsgroups_test.target

# Define the pipeline for text classification
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])

# Define the parameter grid for alpha
# alpha values typically range from 0 (no smoothing) to 1 or more.
# We'll use a logarithmic scale for better exploration.
param_grid = {
    'clf__alpha': np.logspace(-3, 0, 10),  # 10 values between 0.001 and 1
}

# Use StratifiedKFold for cross-validation
# n_splits=5 as requested in the exercise
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("\n--- Exercise 1 & 2: Grid Search ---")

# 1. For the Naive Bayes model, use grid search 5-fold cross-validation
#    across different values of alpha to find the best model.
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='f1_macro', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print(f"Best parameters from Grid Search: {grid_search.best_params_}")
best_alpha_grid = grid_search.best_params_['clf__alpha']
print(f"What value of alpha did you obtain? {best_alpha_grid}")

# 2. For the best value of alpha, compute the f1_macro score on the test set.
y_pred_grid = grid_search.predict(X_test)
f1_macro_grid = f1_score(y_test, y_pred_grid, average='macro')
print(f"What is the model's f1_macro score (Grid Search)? {f1_macro_grid}")


print("\n--- Exercise 3: Random Search ---")

# 3. Repeat Exercise 1 and 2 for random search 5-fold cross-validation
#    across different values of alpha. Compute the f1_macro score on the test set.

# For random search, we'll draw 10 samples from a broader distribution
# or the same distribution, but we typically use more iterations (n_iter).
# Let's use 20 iterations for random search to make it distinct from grid search's 10 points.
param_distributions = {
    'clf__alpha': np.logspace(-3, 0, 100) # Draw from 100 possible values
}

random_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=20, cv=cv,
                                   scoring='f1_macro', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, y_train)

print(f"Best parameters from Random Search: {random_search.best_params_}")
best_alpha_random = random_search.best_params_['clf__alpha']
print(f"What value of alpha did you obtain (Random Search)? {best_alpha_random}")

y_pred_random = random_search.predict(X_test)
f1_macro_random = f1_score(y_test, y_pred_random, average='macro')
print(f"What is the model's f1_macro score (Random Search)? {f1_macro_random}")

# Did you get a better f1_macro score compared to grid search in Exercise 2?
if f1_macro_random > f1_macro_grid:
    print(f"Did you get a better f1_macro score compared to grid search in Exercise 2? Yes, Random Search was better.")
elif f1_macro_random < f1_macro_grid:
    print(f"Did you get a better f1_macro score compared to grid search in Exercise 2? No, Grid Search was better.")
else:
    print(f"Did you get a better f1_macro score compared to grid search in Exercise 2? Both achieved the same f1_macro score.")

Loading 20 Newsgroups dataset...
Dataset loaded.

--- Exercise 1 & 2: Grid Search ---
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters from Grid Search: {'clf__alpha': np.float64(0.004641588833612777)}
What value of alpha did you obtain? 0.004641588833612777
What is the model's f1_macro score (Grid Search)? 0.9448299708006562

--- Exercise 3: Random Search ---
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters from Random Search: {'clf__alpha': np.float64(0.004641588833612782)}
What value of alpha did you obtain (Random Search)? 0.004641588833612782
What is the model's f1_macro score (Random Search)? 0.9448299708006562
Did you get a better f1_macro score compared to grid search in Exercise 2? Both achieved the same f1_macro score.
